In [6]:
import uproot as up
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import vector
#import zfit
import mplhep as hep
import yaml
import seaborn as sns
import particle
import vector
#from CMS_cuts import CMS_cut_func
from uncertainties import ufloat
hep.style.use('ATLAS')
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy
import pandas as pd
intum=3
from sklearn.metrics import roc_curve, auc
import tensorflow as tf
import tensorflow.keras.backend as K
import uncertainties.unumpy as unp

# Function Definitions


In [7]:
def corr_coeff(a,b):
    a_resample=[]
    b_resample=[]
    for i in range(len(unp.nominal_values(a))):
        if unp.nominal_values(a)[i]<0:

            a_bootstrap=np.random.poisson(np.abs(unp.nominal_values(a)[i]),1000)
            a_resample.append(-a_bootstrap)
        else:
            a_bootstrap=np.random.poisson(unp.nominal_values(a)[i],1000)
            a_resample.append(a_bootstrap)
        if unp.nominal_values(b)[i]<0:

            b_bootstrap=np.random.poisson(np.abs(unp.nominal_values(b)[i]),1000)
            b_resample.append(-b_bootstrap)
        else:
            b_bootstrap=np.random.poisson(unp.nominal_values(b)[i],1000)
            b_resample.append(b_bootstrap)
        

    a_resample=np.concatenate(a_resample)
    b_resample=np.concatenate(b_resample)
    cov = np.cov(a_resample, b_resample, ddof=0)[0, 1]
    sigma_a = np.std(a_resample)
    sigma_b = np.std(b_resample)
    rho = cov / (sigma_a * sigma_b)
    return rho
def error_propagation(a,operation,b):
    rho=corr_coeff(a,b)
    a_nom=unp.nominal_values(a)
    b_nom=unp.nominal_values(b)
    a_err=unp.std_devs(a)
    b_err=unp.std_devs(b)


    if operation=="/":
        
        result=a_nom/b_nom

        result_err=np.sqrt(  (a_err/b_nom)**2 +((a_nom*b_err/b_nom**2)**2  ))

        

    if operation=='-':

        result=a_nom-b_nom

        result_err=np.sqrt(a_err**2+b_err**2 - 2*rho*a_err*b_err)



    if operation=='+':
        result=a_nom+b_nom

        result_err=np.sqrt(a_err**2+ b_err**2 + 2*1*a_err*b_err)
    
    return unp.uarray(result,result_err)


def error_propagation_test(a,operation,b,rho):
    a_nom=unp.nominal_values(a)
    b_nom=unp.nominal_values(b)
    a_err=unp.std_devs(a)
    b_err=unp.std_devs(b)


    if operation=="/":
        
        result=a_nom/b_nom

        result_err=np.sqrt(  (a_err/b_nom)**2 +((a_nom*b_err/b_nom**2)**2  ))

        

    if operation=='-':

        result=a_nom-b_nom

        result_err=np.sqrt(a_err**2+b_err**2 - 2*rho*a_err*b_err)
        print(rho)


    if operation=='+':
        result=a_nom+b_nom

        result_err=np.sqrt(a_err**2+ b_err**2 + 2*rho*a_err*b_err)
    
    return unp.uarray(result,result_err)


In [8]:
def to_categorical(labels, num_classes=None):

    # get num_classes from max label if None
    if num_classes is None:
        num_classes = int(np.max(labels)) + 1

    y = np.asarray(labels, dtype=int)
    n = y.shape[0]
    categorical = np.zeros((n, num_classes))

    # index into array and set appropriate values to 1
    categorical[np.arange(n), y] = 1

    return categorical

# Data Imports

In [9]:
sim_det, sim_part, data_det, data_part = pd.read_csv('/Users/claudiumihai/MPhys/Analysis/Semester2/DataFiles/SM/DataSM_Detector_level_new_jet_veto.csv'),\
    pd.read_csv('/Users/claudiumihai/MPhys/Analysis/Semester2/DataFiles/SM/DataSM_Particle_level_new_jet_veto.csv'),\
        pd.read_csv('/Users/claudiumihai/MPhys/Analysis/Semester2/DataFiles/cwtil10/DataEFT_cp_odd_cwtil1_Detector_level.csv'),\
              pd.read_csv('/Users/claudiumihai/MPhys/Analysis/Semester2/DataFiles/cwtil10/DataEFT_cp_odd_cwtil1_Particle_level.csv')
lumi=138
tree=up.open('/Users/claudiumihai/MPhys/Analysis/Semester2/DataFiles/ROOT Files/SM/CMS_500k.root:Delphes')
w_full_sm=tree.arrays(['Event.Weight'],library='ak')
sim_part['Event.Weight']=w_full_sm[sim_part['Unnamed: 0']]['Event.Weight']
sim_const=lumi*0.9962*1000/np.sum(w_full_sm['Event.Weight'])
tree=up.open('/Users/claudiumihai/MPhys/Analysis/Semester2/DataFiles/ROOT Files/EFT/CP_odd/EFT_cp_odd_cwtil1.root:Delphes')
w_full_np1=tree.arrays(['Event.Weight'],library='ak')
data_part['Event.Weight']=w_full_np1[data_part['Unnamed: 0']]['Event.Weight']

data_const=lumi*1.153*1000/np.sum(w_full_np1['Event.Weight'])


# Pre-Processing

In [10]:
sim_det['Lepton.PT']=sim_det.apply(lambda row : row ['Electron.PT'] if row['Electron_size']==1 else row['Muon.PT'],axis=1)
sim_det['Lepton.Eta']=sim_det.apply(lambda row : row ['Electron.Eta'] if row['Electron_size']==1 else row['Muon.Eta'],axis=1)
sim_det['Lepton.Phi']=sim_det.apply(lambda row : row ['Electron.Phi'] if row['Electron_size']==1 else row['Muon.Phi'],axis=1)
sim_part['Lepton.PT']=sim_part.apply(lambda row : row ['Electron.PT'] if row['Electron_size']==1 else row['Muon.PT'],axis=1)
sim_part['Lepton.Eta']=sim_part.apply(lambda row : row ['Electron.Eta'] if row['Electron_size']==1 else row['Muon.Eta'],axis=1)
sim_part['Lepton.Phi']=sim_part.apply(lambda row : row ['Electron.Phi'] if row['Electron_size']==1 else row['Muon.Phi'],axis=1)
data_det['Lepton.PT']=data_det.apply(lambda row : row ['Electron.PT'] if row['Electron_size']==1 else row['Muon.PT'],axis=1)
data_det['Lepton.Eta']=data_det.apply(lambda row : row ['Electron.Eta'] if row['Electron_size']==1 else row['Muon.Eta'],axis=1)
data_det['Lepton.Phi']=data_det.apply(lambda row : row ['Electron.Phi'] if row['Electron_size']==1 else row['Muon.Phi'],axis=1)
data_part['Lepton.PT']=data_part.apply(lambda row : row ['Electron.PT'] if row['Electron_size']==1 else row['Muon.PT'],axis=1)
data_part['Lepton.Eta']=data_part.apply(lambda row : row ['Electron.Eta'] if row['Electron_size']==1 else row['Muon.Eta'],axis=1)
data_part['Lepton.Phi']=data_part.apply(lambda row : row ['Electron.Phi'] if row['Electron_size']==1 else row['Muon.Phi'],axis=1)
data_det=data_det.loc[(data_det['Lepton.PT']<300) &(data_det['Photon.PT']<300)  & (data_det['MissingET.MET']<300)]
data_part=data_part.loc[(data_part['Lepton.PT']<300) &(data_part['Photon.PT']<300)  & (data_part['MissingET.MET']<300)]
sim_det=sim_det.loc[(sim_det['Lepton.PT']<300) &(sim_det['Photon.PT']<300)  & (sim_det['MissingET.MET']<300)]
sim_part=sim_part.loc[(sim_part['Lepton.PT']<300) &(sim_part['Photon.PT']<300)  & (sim_part['MissingET.MET']<300)]


unfold_vars=['Lepton.PT','Lepton.Phi','Lepton.Eta','Photon.PT','Photon.Eta','Photon.Phi','MissingET.MET','MissingET.Phi']


X_det=np.asarray([np.concatenate((data_det[var],sim_det[var])) for var in unfold_vars]).T
Y_det=to_categorical(np.concatenate(( np.ones(len(data_det['Lepton.Eta'])),
                                             np.zeros(len(sim_det['Lepton.Eta']))   )))





X_gen=np.asarray([np.concatenate((sim_part[var],sim_part[var])) for var in unfold_vars]).T
Y_gen=to_categorical(np.concatenate(( np.ones(len(sim_part['Lepton.Eta'])),
                                             np.zeros(len(sim_part['Lepton.Eta']))   )))


ndata, nsim = np.count_nonzero(Y_det[:,1]), np.count_nonzero(Y_det[:,0])
wdata = np.ones(ndata)
winit = ndata/nsim*np.ones(nsim)
sim_det['event']=sim_det['Unnamed: 0']
sim_part['event']=sim_part['Unnamed: 0']
data_det['event']=None
input_det=pd.concat([pd.DataFrame(data_det[unfold_vars+['event']]),pd.DataFrame(sim_det[unfold_vars+['event']])])
input_det['target']=np.concatenate((np.ones(len(data_det)),np.zeros(len(sim_det))))
input_det['w']=np.concatenate((np.ones(len(data_det)),winit*  sim_det['Event.Weight'])  )

input_gen=pd.concat([pd.DataFrame(sim_part[unfold_vars+['event']]),pd.DataFrame(sim_part[unfold_vars+['event']])])
input_gen['target']=np.concatenate((np.ones(len(sim_part)),np.zeros(len(sim_part))))
input_gen['w']=np.concatenate((np.ones(len(sim_part)),np.ones(len(sim_part))))
D=np.array(sim_det['Unnamed: 0'])
P=np.array(sim_part['Unnamed: 0'])                              
full_set_events=np.unique(np.concatenate((D,P)))
common_events=np.intersect1d(D,P)
det_eff_events=np.setdiff1d(P,D)
neutrino_eff_events=np.setdiff1d(D,P)
particle_events=P

def normalize_columns(df, columns):
    df_normalized = df[columns].copy()  # Work only on the specified columns
    for col in columns:
        min_val = df[col].min()
        max_val = df[col].max()
        if max_val - min_val == 0:  # Avoid division by zero
            df_normalized[col] = 0
        else:
            df_normalized[col] = (df[col] - min_val) / (max_val - min_val)
    return df_normalized

def normalize_columns_to_minus1_1(df, columns):
    df_normalized = df[columns].copy()  # Work only on the specified columns
    for col in columns:
        min_val = df[col].min()
        max_val = df[col].max()
        if max_val - min_val == 0:  # Avoid division by zero
            df_normalized[col] = 0
        else:
            # Normalize to [-1, 1]
            df_normalized[col] = 2 * (df[col] - min_val) / (max_val - min_val) - 1
    return df_normalized

In [11]:
import pickle
with open('/Users/claudiumihai/MPhys/Analysis/Semester2/DataFiles/100NN_weights/coin_flip.pkl','rb') as file:
    coin_flip=pickle.load(file)
coin_flip

array([0.45791345, 0.412319  , 0.59548107, ..., 0.82213769, 0.57819755,
       0.750144  ])

In [12]:
data_det['coin_flip']=coin_flip[:len(data_det)]
sim_det['coin_flip']=coin_flip[:len(sim_det)]
sim_part['coin_flip']=coin_flip[:len(sim_part)]
data_part['coin_flip']=coin_flip[:len(data_part)]

# CP-sensitive Observables

In [13]:
def phi_detector(data_df):
    W_boson=particle.Particle.from_pdgid(24)
    M_w=W_boson.mass/1000

    data_df['large_boost']=0.0
    data_df.loc[data_df['Electron_size']==1,'large_boost']=M_w**2/(data_df['Electron.PT']*data_df['MissingET.MET'])
    data_df.loc[data_df['Muon_size']==1,'large_boost']=M_w**2/(data_df['Muon.PT']*data_df['MissingET.MET'])

    data_df['delta']=0.0
    data_df.loc[data_df['Electron_size']==1,'delta']=(M_w**2-data_df['Mt_lv']**2)/(2*data_df['Electron.PT']*data_df['MissingET.MET'])
    data_df.loc[data_df['Muon_size']==1,'delta']=(M_w**2-data_df['Mt_lv']**2)/(2*data_df['Muon.PT']*data_df['MissingET.MET'])


    data_df['lepton_mass']=0.0
    data_df.loc[data_df['Muon_size']==1,'lepton_mass']=particle.Particle.from_pdgid(13).mass/1000
    data_df.loc[data_df['Electron_size']==1,'lepton_mass']=particle.Particle.from_pdgid(11).mass/1000
    data_df['Mt_lv']=0.0


    data_df.loc[data_df['Electron_size']==1,'Mt_lv']=np.sqrt(  
        ( data_df['MissingET.MET']+ np.sqrt((data_df['Electron.PT'])**2+data_df['lepton_mass']**2))**2 
        - (   data_df['Electron.PT']*np.cos(data_df['Electron.Phi']) +data_df['MissingET.MET']*np.cos(data_df['MissingET.Phi'])  )**2
            - (   data_df['Electron.PT']*np.sin(data_df['Electron.Phi']) +data_df['MissingET.MET']*np.sin(data_df['MissingET.Phi'])  )**2   )

    data_df.loc[data_df['Muon_size']==1,'Mt_lv']=np.sqrt(  
        ( data_df['MissingET.MET']+ np.sqrt((data_df['Muon.PT'])**2+data_df['lepton_mass']**2))**2 
        - (   data_df['Muon.PT']*np.cos(data_df['Muon.Phi']) +data_df['MissingET.MET']*np.cos(data_df['MissingET.Phi'])  )**2
            - (   data_df['Muon.PT']*np.sin(data_df['Muon.Phi']) +data_df['MissingET.MET']*np.sin(data_df['MissingET.Phi'])  )**2   )


    data_df['large_boost']=0.0
    data_df.loc[data_df['Electron_size']==1,'large_boost']=M_w**2/(data_df['Electron.PT']*data_df['MissingET.MET'])
    data_df.loc[data_df['Muon_size']==1,'large_boost']=M_w**2/(data_df['Muon.PT']*data_df['MissingET.MET'])

    data_df['delta']=0.0
    data_df.loc[data_df['Electron_size']==1,'delta']=(M_w**2-data_df['Mt_lv']**2)/(2*data_df['Electron.PT']*data_df['MissingET.MET'])
    data_df.loc[data_df['Muon_size']==1,'delta']=(M_w**2-data_df['Mt_lv']**2)/(2*data_df['Muon.PT']*data_df['MissingET.MET'])


    data_df['sign']=np.sign(data_df['delta'])
    data_df.loc[(data_df['sign']==-1.0) & (data_df['Electron_size']==1),'Eta_v']=data_df['Electron.Eta']
    data_df.loc[(data_df['sign']==-1.0) & (data_df['Muon_size']==1),'Eta_v']=data_df['Muon.Eta']

    def eta_based_on_coin_flip_electron(row):
        eta_plus = row['Electron.Eta'] + np.log(1 + np.sqrt(row['delta']) * np.sqrt(2 + row['delta']) + row['delta'])
        eta_minus = row['Electron.Eta'] - np.log(1 + np.sqrt(row['delta']) * np.sqrt(2 + row['delta']) + row['delta'])
        # Select based on coin_flip value
        return eta_plus if row['coin_flip']  <0.5 else eta_minus #np.random.choice([eta_plus,eta_minus])#e

    data_df.loc[(data_df['Electron_size'] == 1) & (data_df['delta'] >= 0), 'Eta_v'] = data_df[
        (data_df['Electron_size'] == 1) & (data_df['delta'] >= 0)
    ].apply(eta_based_on_coin_flip_electron, axis=1)

    def eta_based_on_coin_flip_muon(row):
        eta_plus = row['Muon.Eta'] + np.log(1 + np.sqrt(row['delta']) * np.sqrt(2 + row['delta']) + row['delta'])
        eta_minus = row['Muon.Eta'] - np.log(1 + np.sqrt(row['delta']) * np.sqrt(2 + row['delta']) + row['delta'])
        # Select based on coin_flip value
        return eta_plus if row['coin_flip'] <0.5 else eta_minus #np.random.choice([eta_plus,eta_minus]) #e

    # Apply the updated function
    data_df.loc[(data_df['Muon_size'] == 1) & (data_df['delta'] >= 0), 'Eta_v'] = data_df[
        (data_df['Muon_size'] == 1) & (data_df['delta'] >= 0)
    ].apply(eta_based_on_coin_flip_muon, axis=1)





    def phi_transf(phi):
        return np.piecewise(
        phi,
        [
            phi < -np.pi / 2,  
            np.abs(phi) < np.pi / 2,  
            phi > np.pi / 2  
        ],
        [
            lambda phi: -(np.pi + phi),  
            lambda phi: phi,  
            lambda phi: np.pi - phi  
        ]
    )

    def phi_recon(charge,particle):

        p_l=vector.zip({'pt':data_df[particle+'.PT'].loc[(data_df[particle+'_size']==1) & (data_df[particle+'.Charge']==charge)] ,'eta':data_df[particle+'.Eta'].loc[(data_df[particle+'_size']==1) & (data_df[particle+'.Charge']==charge)],
                        'phi':data_df[particle+'.Phi'].loc[(data_df[particle+'_size']==1) & (data_df[particle+'.Charge']==charge)],'mass':data_df['lepton_mass'].loc[(data_df[particle+'_size']==1) & (data_df[particle+'.Charge']==charge)]  })
        p_y=vector.zip({'pt':data_df['Photon.PT'].loc[(data_df[particle+'_size']==1) & (data_df[particle+'.Charge']==charge)] ,'eta':data_df['Photon.Eta'].loc[(data_df[particle+'_size']==1) & (data_df[particle+'.Charge']==charge)],
                        'phi':data_df['Photon.Phi'].loc[(data_df[particle+'_size']==1) & (data_df[particle+'.Charge']==charge)],'mass':0 })

        p_vl=vector.zip({'pt':data_df['MissingET.MET'].loc[(data_df[particle+'_size']==1) & (data_df[particle+'.Charge']==charge)] ,'eta':data_df['Eta_v'].loc[(data_df[particle+'_size']==1) & (data_df[particle+'.Charge']==charge)],
                    'phi':data_df['MissingET.Phi'].loc[(data_df[particle+'_size']==1) & (data_df[particle+'.Charge']==charge)],'mass':0 })


        p_w=p_l+p_vl
        p_com=p_w+p_y
        p_w_boosted=p_w.boost(-p_com.to_beta3())
        p_l_boosted=p_l.boost(-p_com.to_beta3())
        p_vl_boosted=p_vl.boost(-p_com.to_beta3())

        n1=p_w_boosted.to_Vector3D().cross(-p_com.to_Vector3D())    
        n1=n1/n1.mag

        n_ref=n1.cross(p_w_boosted.to_Vector3D())
        n_ref=n_ref/n_ref.mag
        if charge==1.0:
            n2=-p_l_boosted.to_Vector3D().cross(p_w_boosted.to_Vector3D())
        elif charge==-1.0:
            n2=-p_vl_boosted.to_Vector3D().cross(p_w_boosted.to_Vector3D())
        else: return 'error'

        n2=n2/n2.mag   

        x=n2.dot(n1)
        y=n2.dot(n_ref)

        phi=-np.arctan2(y,x)
        phi=np.array(phi)

        return phi



    data_df['Phi']=0.0

    data_df.loc[(data_df['Electron_size']==1) & (data_df['Electron.Charge']==1.0),'phi']=phi_recon(1.0,'Electron')
    data_df.loc[(data_df['Electron_size']==1) & (data_df['Electron.Charge']==-1.0),'phi']=phi_recon(-1.0,'Electron')

    data_df.loc[(data_df['Muon_size']==1) & (data_df['Muon.Charge']==1.0),'phi']=phi_recon(1.0,'Muon')
    data_df.loc[(data_df['Muon_size']==1) & (data_df['Muon.Charge']==-1.0),'phi']=phi_recon(-1.0,'Muon')

    return data_df

def phi_particle(data_df):


    def phi(phi):
        return np.piecewise(
        phi,
        [
            phi < -np.pi / 2,  
            np.abs(phi) < np.pi / 2,  
            phi > np.pi / 2  
        ],
        [
            lambda phi: -(np.pi + phi),  
            lambda phi: phi,  
            lambda phi: np.pi - phi  
        ]
    )

    def phi_recon(charge,particle):

        p_l=vector.zip({'pt':data_df[particle+'.PT'].loc[(data_df[particle+'_size']==1) & (data_df[particle+'.Charge']==charge)] ,'eta':data_df[particle+'.Eta'].loc[(data_df[particle+'_size']==1) & (data_df[particle+'.Charge']==charge)],
                        'phi':data_df[particle+'.Phi'].loc[(data_df[particle+'_size']==1) & (data_df[particle+'.Charge']==charge)],'mass':data_df['lepton_mass'].loc[(data_df[particle+'_size']==1) & (data_df[particle+'.Charge']==charge)]  })
        p_y=vector.zip({'pt':data_df['Photon.PT'].loc[(data_df[particle+'_size']==1) & (data_df[particle+'.Charge']==charge)] ,'eta':data_df['Photon.Eta'].loc[(data_df[particle+'_size']==1) & (data_df[particle+'.Charge']==charge)],
                        'phi':data_df['Photon.Phi'].loc[(data_df[particle+'_size']==1) & (data_df[particle+'.Charge']==charge)],'mass':0 })

        p_vl=vector.zip({'pt':data_df['MissingET.MET'].loc[(data_df[particle+'_size']==1) & (data_df[particle+'.Charge']==charge)] ,'eta':data_df['MissingET.Eta'].loc[(data_df[particle+'_size']==1) & (data_df[particle+'.Charge']==charge)],
                    'phi':data_df['MissingET.Phi'].loc[(data_df[particle+'_size']==1) & (data_df[particle+'.Charge']==charge)],'mass':0 })


        p_w=p_l+p_vl
        p_com=p_w+p_y
        p_w_boosted=p_w.boost(-p_com.to_beta3())
        p_l_boosted=p_l.boost(-p_com.to_beta3())
        p_vl_boosted=p_vl.boost(-p_com.to_beta3())

        n1=p_w_boosted.to_Vector3D().cross(-p_com.to_Vector3D())    
        n1=n1/n1.mag

        n_ref=n1.cross(p_w_boosted.to_Vector3D())
        n_ref=n_ref/n_ref.mag
        if charge==1.0:
            n2=-p_l_boosted.to_Vector3D().cross(p_w_boosted.to_Vector3D())
        elif charge==-1.0:
            n2=-p_vl_boosted.to_Vector3D().cross(p_w_boosted.to_Vector3D())
        else: return 'error'

        n2=n2/n2.mag   

        x=n2.dot(n1)
        y=n2.dot(n_ref)

        phi=-np.arctan2(y,x)
        phi=np.array(phi)

        return phi



    data_df['Phi']=0.0

    data_df.loc[(data_df['Electron_size']==1) & (data_df['Electron.Charge']==1.0),'phi_true']=phi_recon(1.0,'Electron')
    data_df.loc[(data_df['Electron_size']==1) & (data_df['Electron.Charge']==-1.0),'phi_true']=phi_recon(-1.0,'Electron')

    data_df.loc[(data_df['Muon_size']==1) & (data_df['Muon.Charge']==1.0),'phi_true']=phi_recon(1.0,'Muon')
    data_df.loc[(data_df['Muon_size']==1) & (data_df['Muon.Charge']==-1.0),'phi_true']=phi_recon(-1.0,'Muon')

    return data_df

def dPhi_l_y(data_df):
    for i in data_df.index:
        if data_df['Lepton.Eta'][i]>data_df['Photon.Eta'][i]:
            temp=data_df['Lepton.Phi'][i]-data_df['Photon.Phi'][i]
            if temp>np.pi:
                data_df.at[i,'dPhi']=temp-np.pi
            elif temp<-np.pi:
                data_df.at[i,'dPhi']=temp+np.pi
            else:
                data_df.at[i,'dPhi']=temp
        else:
            temp=data_df['Photon.Phi'][i]-data_df['Lepton.Phi'][i]
            if temp>np.pi:
                data_df.at[i,'dPhi']=temp-np.pi
            elif temp<-np.pi:
                 data_df.at[i,'dPhi']=temp+np.pi
            else:
                 data_df.at[i,'dPhi']=temp
            
    return data_df

In [14]:
sim_part=phi_detector(sim_part)
sim_det=phi_detector(sim_det)

data_part=phi_detector(data_part)
data_det=phi_detector(data_det)


sim_part=phi_particle(sim_part)
data_part=phi_particle(data_part)



sim_det=dPhi_l_y(sim_det)
sim_part=dPhi_l_y(sim_part)
data_det=dPhi_l_y(data_det)
data_part=dPhi_l_y(data_part)

# Base Omnifold

In [15]:
unfold_vars=['Lepton.Phi','Lepton.Eta','Photon.Phi','Photon.Eta','MissingET.Phi','Lepton.PT','MissingET.MET','Photon.PT','Eta_v']
unfold_vars_norm=[i+'_std' for i in unfold_vars]

weights_push=np.ones(len(sim_part['Lepton.PT']))
weights_pull=np.ones(len(sim_part['Lepton.PT']))


data_det[unfold_vars_norm]  = normalize_columns(data_det,unfold_vars)
sim_det[unfold_vars_norm]  = normalize_columns(sim_det,unfold_vars)
data_part[unfold_vars_norm]  = normalize_columns(data_part,unfold_vars)
sim_part[unfold_vars_norm]  = normalize_columns(sim_part,unfold_vars)


input_det=pd.concat([pd.DataFrame(data_det[unfold_vars_norm+['event']]),pd.DataFrame(sim_det[unfold_vars_norm+['event']])])
input_det['target']=np.concatenate((np.ones(len(data_det)),np.zeros(len(sim_det))))

input_gen=pd.concat([pd.DataFrame(sim_part[unfold_vars_norm+['event']]),pd.DataFrame(sim_part[unfold_vars_norm+['event']])])
input_gen['target']=np.concatenate((np.ones(len(sim_part)),np.zeros(len(sim_part))))


n=len(unfold_vars)


inputs_step1 = Input((n, ))
hidden_layer_1_step1 = Dense(100, activation='relu')(inputs_step1)
hidden_layer_2_step1 = Dense(100, activation='relu')(hidden_layer_1_step1)
hidden_layer_3_step1 = Dense(100, activation='relu')(hidden_layer_2_step1)

outputs_step1 = Dense(1, activation='sigmoid')(hidden_layer_1_step1)
model_step1 = Model(inputs=inputs_step1, outputs=outputs_step1)


weights_push=np.ones(len(full_set_events))
weights_pull=np.ones(len(full_set_events))

push_mapping=dict(zip(full_set_events,weights_push))
pull_mapping=dict(zip(full_set_events,weights_pull))




inputs_step2 = Input((n, ))
hidden_layer_1_step2 = Dense(100, activation='relu')(inputs_step2)
hidden_layer_2_step2 = Dense(100, activation='relu')(hidden_layer_1_step2)
hidden_layer_3_step2 = Dense(100, activation='relu')(hidden_layer_2_step2)

outputs_step2 = Dense(1, activation='sigmoid')(hidden_layer_3_step2)
model_step2 = Model(inputs=inputs_step2, outputs=outputs_step2)

push_series=pd.Series(push_mapping)
pull_series=pd.Series(pull_mapping)
push_series.loc[D]=np.array(sim_det['Event.Weight'])*winit
data_weights=np.array(data_det['Event.Weight'])
for i in range(5):

    # Step 1

    w1=np.concatenate((data_weights,push_series.loc[D]))

    model_step1 = Model(inputs=inputs_step1, outputs=outputs_step1)
    X_train,X_test,y_train,y_test,w_train,w_test=train_test_split(input_det[unfold_vars_norm],
                                                                input_det['target'],
                                                                w1)

    model_step1.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

    model_step1.fit(X_train,y_train,sample_weight=w_train,epochs=50,batch_size=1500,validation_data=(X_test,y_test,w_test),verbose=False)

    f=model_step1.predict(sim_det.loc[sim_det['event'].isin(common_events)][unfold_vars_norm],batch_size=700)
    
    temp=f/(1.-f)
    temp=np.squeeze(np.nan_to_num(temp)) 

    pull_series.loc[common_events]=temp*push_series.loc[common_events]


    # estimation 1

    estimation_1_X=pd.concat([sim_part.loc[sim_part['event'].isin(common_events)][unfold_vars_norm],sim_part.loc[sim_part['event'].isin(common_events)][unfold_vars_norm]])

    estimation_1_Y=np.concatenate((np.ones(len(sim_part.loc[sim_part['event'].isin(common_events)][unfold_vars_norm])), np.zeros(len(sim_part.loc[sim_part['event'].isin(common_events)][unfold_vars_norm]))))
    
    estimation_1_w=np.concatenate((pull_series.loc[common_events],np.ones(len(sim_part.loc[sim_part['event'].isin(common_events)][unfold_vars_norm]))))

    X_t, X_v, y_t, y_v, w_t, w_v = train_test_split(estimation_1_X,estimation_1_Y,estimation_1_w)

    model_estimation_1=Model(inputs=inputs_step1, outputs=outputs_step1)
    model_estimation_1.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

    model_estimation_1.fit(X_t,y_t,sample_weight=w_t,epochs=50,batch_size=700,validation_data=(X_v,y_v,w_v),verbose=False)

    f=model_estimation_1.predict(sim_part.loc[sim_part['event'].isin(det_eff_events)][unfold_vars_norm],batch_size=400)

    temp=f/(1.-f)

    temp=np.squeeze(np.nan_to_num(temp)) 

    pull_series.loc[det_eff_events]=temp

    # Step 2
    w2=np.concatenate((pull_series[P],push_series[P]))

    model_step2 = Model(inputs=inputs_step2, outputs=outputs_step2)

    X_train,X_test,y_train,y_test,w_train,w_test=train_test_split(input_gen[unfold_vars_norm],
                                                                input_gen['target'],
                                                                w2)
    model_step2.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

    model_step2.fit(X_train,y_train,sample_weight=w_train,epochs=50,batch_size=2000,validation_data=(X_test,y_test,w_test),verbose=False)


    
    f=model_step2.predict(sim_part[unfold_vars_norm],batch_size=1000)
    
    temp=f/(1.-f)

    temp=np.squeeze(np.nan_to_num(temp)) 
    
    
    push_series.loc[P]=temp*push_series[P]


    # Estimation 2

    estimation_2_X=pd.concat([sim_det.loc[sim_det['event'].isin(common_events)][unfold_vars_norm],sim_det.loc[sim_det['event'].isin(common_events)][unfold_vars_norm]])

    estimation_2_Y=np.concatenate((np.ones(len(sim_det.loc[sim_det['event'].isin(common_events)][unfold_vars_norm])), np.zeros(len(sim_det.loc[sim_det['event'].isin(common_events)][unfold_vars_norm]))))
    
    estimation_2_w=np.concatenate((push_series.loc[common_events],np.ones(len(sim_det.loc[sim_det['event'].isin(common_events)][unfold_vars_norm]))))

    X_t, X_v, y_t, y_v, w_t, w_v = train_test_split(estimation_2_X,estimation_2_Y,estimation_2_w)

    model_estimation_2=Model(inputs=inputs_step2, outputs=outputs_step2)
    model_estimation_2.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

    model_estimation_2.fit(X_t,y_t,sample_weight=w_t,epochs=50,batch_size=700,validation_data=(X_v,y_v,w_v),verbose=False)

    f=model_estimation_2.predict(sim_det.loc[sim_det['event'].isin(neutrino_eff_events)][unfold_vars_norm],batch_size=50)

    temp=f/(1.-f)

    temp=np.squeeze(np.nan_to_num(temp)) 

    push_series.loc[neutrino_eff_events]=temp

    # Output
w_push=np.array(push_series.loc[P])

109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 406us/step
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 372us/step
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 821us/step
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 398us/step
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 423us/step
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 416us/step
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 848us/step
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 424us/step
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 443us/step
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 370us/step
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 798us/step
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 402us/step
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 448us/step
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 391us/step
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 881us/step
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 407us/step
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 416us/step
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 373us/step
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 797us/step
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 405us/step


In [16]:
weights_to_test=[w_push]

# Performance Metrics

In [31]:
unfold_vars=['Lepton.Phi','Lepton.Eta','Photon.Phi','Photon.Eta','MissingET.Phi','Lepton.PT','MissingET.MET','Photon.PT','Eta_v']
all_vars=unfold_vars+['phi','dPhi']
scores={key: {} for key in all_vars}


for w in weights_to_test:
    sim_part['omni']=w
    data_sets={'SM':{'particle':sim_part,'detector':sim_det,'weights_detector':sim_det['Event.Weight'],'weights_particle':sim_part['Event.Weight'],'constant':sim_const,'weights_full':w_full_sm},
           'EFT_NP1':{'particle':data_part,'detector':data_det,'weights_detector':data_det['Event.Weight'],'weights_particle':data_part['Event.Weight'],'constant':data_const,'weights_full':w_full_np1},}
    sim_part_copy=sim_part.loc[
    (sim_part['Lepton.PT'] > 0) & (sim_part['Lepton.PT'] <300) &
    (sim_part['Photon.PT'] > 0) & (sim_part['Photon.PT'] <300) &
    (sim_part['MissingET.MET'] > 0) & (sim_part['MissingET.MET'] <300)
    ].copy()


    sim_det_copy=sim_det.loc[
        (sim_det['Lepton.PT'] > 0) & (sim_det['Lepton.PT'] <300) &
        (sim_det['Photon.PT'] > 0) & (sim_det['Photon.PT'] <300) &
        (sim_det['MissingET.MET'] > 0) & (sim_det['MissingET.MET'] <300)
    ].copy()


    data_part_copy=data_part.loc[
        (data_part['Lepton.PT'] > 0) & (data_part['Lepton.PT'] <300) &
        (data_part['Photon.PT'] > 0) & (data_part['Photon.PT'] <300) &
        (data_part['MissingET.MET'] > 0) & (data_part['MissingET.MET'] <300)
    ].copy()



    data_det_copy=data_det.loc[
        (data_det['Lepton.PT'] > 0) & (data_det['Lepton.PT'] <300) &
        (data_det['Photon.PT'] > 0) & (data_det['Photon.PT'] <300) &
        (data_det['MissingET.MET'] > 0) & (data_det['MissingET.MET'] <300)
    ].copy()




    data_sets={'SM':{'particle':sim_part_copy,'detector':sim_det_copy,'weights_detector':sim_det_copy['Event.Weight'],'weights_particle':sim_part_copy['Event.Weight'],'constant':sim_const,'weights_full':w_full_sm},
            'EFT_NP1':{'particle':data_part_copy,'detector':data_det_copy,'weights_detector':data_det_copy['Event.Weight'],'weights_particle':data_part_copy['Event.Weight'],'constant':data_const,'weights_full':w_full_np1},}


    from hist_plot import hist_plot

    scores_temp={}
    for var in all_vars:
        

        sim_det_temp=data_sets['SM']['detector'][var]
        sim_part_temp=data_sets['SM']['particle'][var]
        data_det_temp=data_sets['EFT_NP1']['detector'][var]
        data_part_temp=data_sets['EFT_NP1']['particle'][var]

        omni_weights=data_sets['SM']['particle']['omni']*sim_const
        result_data_det_temp=hist_plot(data=data_det_temp,weights=data_sets['EFT_NP1']['weights_detector'],bins=20,constant=data_sets['EFT_NP1']['constant'],weights_full=data_sets['EFT_NP1']['weights_full'])

        
        result_sim_part_temp=hist_plot(data=sim_part_temp,weights=data_sets['SM']['weights_particle'],bins=20,constant=data_sets['SM']['constant'],weights_full=data_sets['SM']['weights_full'])


        result_data_part_temp=hist_plot(data=data_part_temp,weights=data_sets['EFT_NP1']['weights_particle'],bins=20,constant=data_sets['EFT_NP1']['constant'],weights_full=data_sets['EFT_NP1']['weights_full'])

        result_sim_part_temp_omni=hist_plot(data=sim_part_temp,weights=data_sets['SM']['particle']['omni'],bins=20,constant=data_sets['SM']['constant'],weights_full=data_sets['SM']['weights_full'])

        metric_omni=error_propagation(result_data_part_temp['expected_norm'],'/',result_sim_part_temp_omni['expected_norm'])

        metric=error_propagation(result_data_part_temp['expected_norm'],'/',result_sim_part_temp['expected_norm'])
    
        #if (var=='dPhi') or (var=='phi'):
        chi=np.sum( (np.square(unp.nominal_values(metric)-1))/unp.std_devs(metric) )
        chi_omni=np.sum( (np.square(unp.nominal_values(metric_omni)-1))/unp.std_devs(metric_omni) )
        scores_temp[var]=[chi,chi_omni]

        omni_iter='omni'+'_'+str(weights_to_test.index(w))

        scores[var]['base']=chi
        scores[var][omni_iter]=chi_omni

    #if (scores['dPhi'][-1]<base_score['dPhi']*0.2)  and  (scores['phi'][-1]<base_score['phi']*0.2)    :
    #   good_weights.append(w)
        


In [32]:
scores

{'Lepton.Phi': {'base': 0.4325475007528058, 'omni_0': 0.49982478966705185},
 'Lepton.Eta': {'base': 0.20996827122821105, 'omni_0': 0.5936189607938415},
 'Photon.Phi': {'base': 0.578695465583572, 'omni_0': 0.5419955563317573},
 'Photon.Eta': {'base': 0.3879829210827911, 'omni_0': 0.9229497435543266},
 'MissingET.Phi': {'base': 0.2893657934529643, 'omni_0': 0.42096583468738213},
 'Lepton.PT': {'base': 124.86666674779167, 'omni_0': 3.421142665865231},
 'MissingET.MET': {'base': 159.7886771425295, 'omni_0': 3.535423210421304},
 'Photon.PT': {'base': 285.4410635028072, 'omni_0': 1.038791798401853},
 'Eta_v': {'base': 6.650360050344657, 'omni_0': 3.2322755506240988},
 'phi': {'base': 4.174137002693565, 'omni_0': 0.7130374885774639},
 'dPhi': {'base': 28.02647185567154, 'omni_0': 0.6321273659917592}}

In [33]:
scores_df=pd.DataFrame(scores)
scores_df

,Lepton.Phi,Lepton.Eta,Photon.Phi,Photon.Eta,MissingET.Phi,Lepton.PT,MissingET.MET,Photon.PT,Eta_v,phi,dPhi
base,0.432548,0.209968,0.578695,0.387983,0.289366,124.866667,159.788677,285.441064,6.650360,4.174137,28.026472
omni_0,0.499825,0.593619,0.541996,0.922950,0.420966,3.421143,3.535423,1.038792,3.232276,0.713037,0.632127
